In [87]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

# Preprocess

In [88]:
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

path = 'barkers.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

corpus length: 1857901


# Compute Character Indices

In [89]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 1608


# Vectorize Sentences

In [90]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 619287
Vectorization...


# Create Simple Model

In [91]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...


# Define Callback

In [92]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Train

In [93]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

Epoch 1/60
619287/619287 [==============================] - 510s 824us/step - loss: 3.0851

----- Generating text after Epoch: 0
----- Generating with seed: "라는 베르반님의 지시에......” 커크는 민망한 듯 말끝을 흘렸다. 서"
라는 베르반님의 지시에......” 커크는 민망한 듯 말끝을 흘렸다. 서연은 아무 도느의 말에 서연은 이 정도에 대한 것이다. 그리고 표정을 지으며 말을 하고 있었다. 그 자리는 아리엘이 다른 사람들은 무시를 받아는 것을 만들었다. 그 자신의 모습을 보고 있는지 나타는 것이라는 것은 다른 이대로 그 사람들은 그 서연은 서연을 바라보았다. 그들이 서연을 바라보았다. "아... 하지만 비교 수 있는 일이야. 그리고 한 명 가 아무 수도 없었다. "정말 거야. 그 사람들은 서연님이 지나 마주를 둘러보았다. 서연은 모두 있는 사람들이 다른 것 같지 않았다. 그 정도는 화려하지 않는 것이라는 것이 있었다. 그리고 정신의 일이 있는 상황에서 살고 있는 것은 아무  상도 없었다. 그 그에는 지금까지 바라보는 서연의 보고 자신의 대한 마나를 막아 보았다. 그들이 자신의 몸을 물었다. 그런데 
Epoch 2/60
619287/619287 [==============================] - 513s 828us/step - loss: 2.4530

----- Generating text after Epoch: 1
----- Generating with seed: "수스가 바론시아 기사단에서 최정예를 상대하고 있는 것 역시 발견했다. 땅"
수스가 바론시아 기사단에서 최정예를 상대하고 있는 것 역시 발견했다. 땅을 이야기 하는 분위기기 시작했다. 그런데 이 사람은 그 소리가 들어갔다. "아............................" "아................................" "........

619287/619287 [==============================] - 511s 825us/step - loss: 0.9395

----- Generating text after Epoch: 25
----- Generating with seed: " 지배했다. 뿌드득. 로니스는 서연이 사라진 곳을 바라보며 이를 갈았다."
 지배했다. 뿌드득. 로니스는 서연이 사라진 곳을 바라보며 이를 갈았다. 하지만 그들은 눈앞에 있는 남자를 보며 입을 열었다. "여러분과 그 아... 그렇다면 우리가 그런 전였는지 모르겠군. 지금 같은 남자에게 여자가 드래곤에게 멜리언님이 칼스테인 공작님이 제게 말해주신 것이라면 어떻게 해주라고 제어 하는 것이 아겠습니까?" "......" 로니스의 방안에 검을 내려던 그녀들의 눈동자가 살짝 흔들렸다. "소리는 영지를 공격했다. 물건......" "입뭐 다라!" "......" "......" 서연은 잠시 어곳이 이스반 국왕과 목소리리리가 하하는 듯 말했다. "이제 그만 일세." 친구인 라샤드를 응시했다. “......” “내가 사실 처음은 아니고 이번 전쟁에서 가 지나” 오늘도 오시하고 있지 않았다. "아... 점점 알고 있는군요." "......" 서연은 칼베리안의 말에 서
Epoch 27/60
619287/619287 [==============================] - 511s 825us/step - loss: 0.9013

----- Generating text after Epoch: 26
----- Generating with seed: "맞물리는 것이 분명합니다." "호오~" 또다시 묘한 소리로 리액션을 보이"
맞물리는 것이 분명합니다." "호오~" 또다시 묘한 소리로 리액션을 보이며 마나를 갑자였다. 서연은 피식 웃음을 흘린 채 여자들을 바라보았다. 그리고 로비에 입을 기대고 있었다. 서연과 일행들은 말을 이었다. "칼스테인 공작님의 말을 들은 다른 분의 사람들이 정말 아름답고 그들을 이끌고 했던 대로에

/home/vcubuntu2/tf/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


로비에서 당하고 있는 무도회가 나리는 것 네 미르는 조금 전에 달라오는 생각이 들었다. 자신들이 어져 있는 하나만을 보다 더 이상 전심하는 것을 바라보고 있는 것이 이곳에 나서고 있었다. 제 이름 한 번의 어쌔신들은 어 대한 보였다. 그런데 누라타는 같은 상황에 아리엘과 다른 사람들이 지금 밖으로 나갔다. "칼스테인영스." 서연의 말에 서연은 잠시 생각에 잠겨 있다. "가 =듯한 남자가 그렇게 말하고 있는데 제가 그대로 다가오지 않았어? 그리고 이 정도 없는 것 같습니다. 서연의 말에 사람들은 모두 고개를 숙였다. "아... 그에 대한 참장을 할 수 있다고 하라고 마나? 지금 모두 공주지 않은 곳으로 돌아가 … 난 얼굴이 분명 저러졌을 바라." "......" "그런데
Epoch 36/60
619287/619287 [==============================] - 508s 820us/step - loss: 8.6384

----- Generating text after Epoch: 35
----- Generating with seed: ". 그들이라면 아무런 문제가 없을 겁니다." 포비르는 안쓰럽게 온몸을 떨"
. 그들이라면 아무런 문제가 없을 겁니다." 포비르는 안쓰럽게 온몸을 떨지 사라이 의고 만었는 다에 것도 으려 로에 사니   우지 대고 들은..  간 . 그 이 정 기 이말다. 로가만로 그으 대는 라도 한자 려까 로들이 고작 려니연  이는 한 도스... 마나 서  에 우으에.도대는  어에 스수두고지 만이 아려 .  기사 사연라는 로 했간에만 은 것  다나 시. ...의마도일 시리게소을 그은안서이 리리자 은이. 연으에 그라  이게 다에  리 보의 다에  게 보. 다. 시 도는 던 나 있는 한 시에 다기도이도 서 에 다히 가  사아한던 시소 서었이 한사를 해.  기에 가  은라도 았인다 스를 한다를 다은 의 말 라 사. 가나 려 보 이에 .에 로려니를 을지고 에 려소는 도 이이고 에 한이.. 게한거보에 는 대에 이게 는 던 려었..는 그습만한 

 내공이 부족해~ =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+noieleitileieleil.co me=)= = = = = 작 가i9 추 회]  회 웠5회목의 회] 날 굴 회 추 이i운c의x e 추었다. 그리고 내가 이스반 국왕을  추고 있었다. 란시아 이스반 국1와 함께 자신의 머리를 자네가 이야기만했던 것을 내려지는 듯 한쪽 이유를 이러한 듯 자신의 모습을 보며 한명에게 생각지도 않았다. 이스반 왕국의 마나를 생각하는 자신의 모습을 보고 고개를 돌렸다. 자신이 있는 자체가 지켜보고 있을 자신의 이스반 국왕을 하는 생각에 살이기 시작했다. 그곳을 지켜보는 자신의 이스반 국왕을 한 명이 지켜보고 있던 마나를 부한 듯한 표정을 지었다. "이 길을 만났던 이스반 국왕을 보이지만 자네가 자신을 지려고 이러한 것이 아니었다. 그 서연을 부리며 자신의 이야기
Epoch 49/60
619287/619287 [==============================] - 504s 814us/step - loss: 2.7585

----- Generating text after Epoch: 48
----- Generating with seed: " 살짝 고개를 끄덕였다. “틀리진 않습니다. 제 가문에 내려오는 특수한 "
 살짝 고개를 끄덕였다. “틀리진 않습니다. 제 가문에 내려오는 특수한 고개를 끄덕였다. “그렇다면 그리고 가지에 않았다. 그러나 어니 했다. 지금의 기사들에게 말하지................................" "그렇다면 자신의 말에 말을 하는지... "오......" "않습니다." "그렇게 이렇게 말을 했다. 지금의 마음에 있는 모든 것이다. "어슨 말을 가으어나지 못했다. 그 고개를 끄덕였다. “못하................." "조금 이어. 그리고 다가가지......" "그렇다면 그 가지고 있다. 그 “무다가가 다음에 사람들은 무시적으로 오기의 보고 있었다. 그리고 지금 자신이 말했다. “그대다!" "어 지